In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# pull in data sourced by Madina and replicate cleaning steps below
meteor_data = pd.read_csv('Resources/Meteorite_Landings.csv')

In [3]:
meteor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         45716 non-null  object 
 1   id           45716 non-null  int64  
 2   nametype     45716 non-null  object 
 3   recclass     45716 non-null  object 
 4   mass (g)     45585 non-null  float64
 5   fall         45716 non-null  object 
 6   year         45425 non-null  float64
 7   reclat       38401 non-null  float64
 8   reclong      38401 non-null  float64
 9   GeoLocation  38401 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 3.5+ MB


In [4]:
meteor_data.describe()

,id,mass (g),year,reclat,reclong
count,45716.000000,4.558500e+04,45425.000000,38401.000000,38401.000000
mean,26889.735104,1.327808e+04,1991.828817,-39.122580,61.074319
std,16860.683030,5.749889e+05,25.052766,46.378511,80.647298
min,1.000000,0.000000e+00,860.000000,-87.366670,-165.433330
25%,12688.750000,7.200000e+00,1987.000000,-76.714240,0.000000
50%,24261.500000,3.260000e+01,1998.000000,-71.500000,35.666670
75%,40656.750000,2.026000e+02,2003.000000,0.000000,157.166670
max,57458.000000,6.000000e+07,2101.000000,81.166670,354.473330


In [5]:
# Dropping null values
meteor_data=meteor_data.dropna()

In [6]:
# Checking missing values. 
meteor_data.isnull().sum()

name           0
id             0
nametype       0
recclass       0
mass (g)       0
fall           0
year           0
reclat         0
reclong        0
GeoLocation    0
dtype: int64

In [7]:
# Find max year
print(meteor_data["year"].max())
meteor_data['year']=meteor_data["year"].astype(int)

2101.0


In [8]:
# Remove year 2101
meteor_data.drop(meteor_data.loc[meteor_data['year']==2101].index, inplace=True)

In [9]:
print(meteor_data.where(meteor_data["year"] > 2023).value_counts().sum())

0


In [10]:
meteor_data.head(15)

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4239.0,Fell,1919,32.10000,71.80000,"(32.1, 71.8)"
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.0,Fell,1949,44.83333,95.16667,"(44.83333, 95.16667)"
7,Agen,392,Valid,H5,30000.0,Fell,1814,44.21667,0.61667,"(44.21667, 0.61667)"
8,Aguada,398,Valid,L6,1620.0,Fell,1930,-31.60000,-65.23333,"(-31.6, -65.23333)"
9,Aguila Blanca,417,Valid,L,1440.0,Fell,1920,-30.86667,-64.55000,"(-30.86667, -64.55)"


In [11]:
# Getting Valids from nametype for map
valids = meteor_data.groupby('nametype').get_group('Valid').copy()
valids.dropna(inplace=True)
valids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38093 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         38093 non-null  object 
 1   id           38093 non-null  int64  
 2   nametype     38093 non-null  object 
 3   recclass     38093 non-null  object 
 4   mass (g)     38093 non-null  float64
 5   fall         38093 non-null  object 
 6   year         38093 non-null  int32  
 7   reclat       38093 non-null  float64
 8   reclong      38093 non-null  float64
 9   GeoLocation  38093 non-null  object 
dtypes: float64(3), int32(1), int64(1), object(5)
memory usage: 3.1+ MB


In [12]:
# Create a scatter map using the 'reclat' and 'reclong' columns of the dataframe as latitude and longitude, respectively
fig = px.scatter_mapbox(meteor_data, lat="reclat", lon="reclong")

# customize the layout of the Map
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=0, lon=0), zoom=0),
    margin=dict(l=0, r=0, t=0, b=0),
    width=600, height=600
)

# Display the map
fig.show()

In [13]:
meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)"
...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990,29.03700,17.01850,"(29.037, 17.0185)"
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999,13.78333,8.96667,"(13.78333, 8.96667)"
45713,Zlin,30410,Valid,H4,3.3,Found,1939,49.25000,17.66667,"(49.25, 17.66667)"
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003,49.78917,41.50460,"(49.78917, 41.5046)"


In [20]:
meteor_data.dtypes

name            object
id               int64
nametype        object
recclass        object
mass (g)       float64
fall            object
year             int32
reclat         float64
reclong        float64
GeoLocation     object
dtype: object

In [26]:
#  try to find info for minnesota meteorite
chrisq = meteor_data.loc[(meteor_data['reclat']==44.60000) & (meteor_data['reclong']==-94.10000)]
chrisq

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
3452,Arlington,2333,Valid,"Iron, IIE-an",8940.0,Found,1894,44.6,-94.1,"(44.6, -94.1)"


In [16]:
meteor_data['fall'].value_counts()

Found    37049
Fell      1065
Name: fall, dtype: int64

end of Madina's data cleaning and analyses

In [15]:
# save cleaned data to csv
# meteor_data.to_csv('Resources/Meteorite_Landings_clean.csv')

In [119]:
# !pip install geopy

In [28]:
import geopy
from geopy.geocoders import Nominatim

In [97]:
meteor_data_co = meteor_data.copy()
meteor_data_co['county'] = 'None_Found'
meteor_data_co['state'] = 'None_Found'
meteor_data_co['country'] = 'None_Found'
# meteor_data_co['elevation'] = 0.0
meteor_data_co

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",None_Found,None_Found,None_Found
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",None_Found,None_Found,None_Found
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",None_Found,None_Found,None_Found
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)",None_Found,None_Found,None_Found
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)",None_Found,None_Found,None_Found
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990,29.03700,17.01850,"(29.037, 17.0185)",None_Found,None_Found,None_Found
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999,13.78333,8.96667,"(13.78333, 8.96667)",None_Found,None_Found,None_Found
45713,Zlin,30410,Valid,H4,3.3,Found,1939,49.25000,17.66667,"(49.25, 17.66667)",None_Found,None_Found,None_Found
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003,49.78917,41.50460,"(49.78917, 41.5046)",None_Found,None_Found,None_Found


In [120]:
# this might not include alaska and the hawaiian islands
us_meteor_data = meteor_data_co.loc[(meteor_data['reclat']>18)&(meteor_data['reclat']<72)&(meteor_data['reclong']>-170)&(meteor_data['reclong']<-67)].copy()
us_meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",None_Found,None_Found,None_Found
28,Allegan,2276,Valid,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",None_Found,None_Found,None_Found
29,Allende,2278,Valid,CV3,2000000.0,Fell,1969,26.96667,-105.31667,"(26.96667, -105.31667)",None_Found,None_Found,None_Found
34,Andover,2295,Valid,L6,3200.0,Fell,1898,44.61667,-70.75000,"(44.61667, -70.75)",None_Found,None_Found,None_Found
46,Archie,2329,Valid,H6,5070.0,Fell,1932,38.50000,-94.30000,"(38.5, -94.3)",None_Found,None_Found,None_Found
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45693,Zaffra,30383,Valid,"Iron, IAB-MG",3000.0,Found,1919,35.00000,-94.75000,"(35.0, -94.75)",None_Found,None_Found,None_Found
45697,Zapaliname,30392,Valid,"Iron, IAB-MG",85000.0,Found,1998,25.00944,-100.75000,"(25.00944, -100.75)",None_Found,None_Found,None_Found
45699,Zapotitlán Salinas,30394,Valid,L4,27.7,Found,1984,18.33333,-97.50000,"(18.33333, -97.5)",None_Found,None_Found,None_Found
45703,Zenda,30400,Valid,"Iron, IAB complex",3700.0,Found,1955,42.51333,-88.48944,"(42.51333, -88.48944)",None_Found,None_Found,None_Found


In [121]:
# https://geopy.readthedocs.io/en/stable/
# https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/

#  initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

for index, row in us_meteor_data.iterrows():
    # get latitude, longitude from the DataFrame

    rowlat = str(row['reclat'])
    rowlon = str(row['reclong'])
    rowloc = geolocator.reverse(rowlat+","+rowlon)
    rowaddress = rowloc.raw['address']
    rowstate = rowaddress.get('state','')
    rowcounty = rowaddress.get('county','')
    rowcountry = rowaddress.get('country','')

    
    try:
        us_meteor_data.loc[index, "county"] = rowcounty
        us_meteor_data.loc[index, "state"] = rowstate
        us_meteor_data.loc[index, "country"] = rowcountry
    except (KeyError, IndexError):
        # If no county is found, set the county as no county found".
        us_meteor_data.loc[index, "county"] = "None_Found"
        us_meteor_data.loc[index, "state"] = "None_Found"
        us_meteor_data.loc[index, "country"] = "None_Found"

us_meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",Thorhild County,Alberta,Canada
28,Allegan,2276,Valid,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",Allegan County,Michigan,United States
29,Allende,2278,Valid,CV3,2000000.0,Fell,1969,26.96667,-105.31667,"(26.96667, -105.31667)",Allende,Chihuahua,México
34,Andover,2295,Valid,L6,3200.0,Fell,1898,44.61667,-70.75000,"(44.61667, -70.75)",Oxford County,Maine,United States
46,Archie,2329,Valid,H6,5070.0,Fell,1932,38.50000,-94.30000,"(38.5, -94.3)",Cass County,Missouri,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45693,Zaffra,30383,Valid,"Iron, IAB-MG",3000.0,Found,1919,35.00000,-94.75000,"(35.0, -94.75)",Le Flore County,Oklahoma,United States
45697,Zapaliname,30392,Valid,"Iron, IAB-MG",85000.0,Found,1998,25.00944,-100.75000,"(25.00944, -100.75)",Galeana,Nuevo León,México
45699,Zapotitlán Salinas,30394,Valid,L4,27.7,Found,1984,18.33333,-97.50000,"(18.33333, -97.5)",Zapotitlán,Puebla,México
45703,Zenda,30400,Valid,"Iron, IAB complex",3700.0,Found,1955,42.51333,-88.48944,"(42.51333, -88.48944)",Walworth County,Wisconsin,United States


In [122]:
# Create a scatter map using the 'reclat' and 'reclong' columns of the dataframe as latitude and longitude, respectively
fig = px.scatter_mapbox(us_meteor_data, lat="reclat", lon="reclong")

# customize the layout of the Map
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=0, lon=0), zoom=0),
    margin=dict(l=0, r=0, t=0, b=0),
    width=600, height=600
)

# Display the map
fig.show()

In [123]:
us_meteor_data = us_meteor_data.loc[us_meteor_data['country']=='United States'].copy()
us_meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country
28,Allegan,2276,Valid,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",Allegan County,Michigan,United States
34,Andover,2295,Valid,L6,3200.0,Fell,1898,44.61667,-70.75000,"(44.61667, -70.75)",Oxford County,Maine,United States
46,Archie,2329,Valid,H6,5070.0,Fell,1932,38.50000,-94.30000,"(38.5, -94.3)",Cass County,Missouri,United States
49,Ash Creek,48954,Valid,L6,9500.0,Fell,2009,31.80500,-97.01000,"(31.805, -97.01)",McLennan County,Texas,United States
54,Athens,4885,Valid,LL6,265.0,Fell,1933,34.75000,-87.00000,"(34.75, -87.0)",Limestone County,Alabama,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45688,Yucca 016,57158,Valid,H5,25.9,Found,2011,34.82658,-114.27763,"(34.82658, -114.27763)",Mohave County,Arizona,United States
45689,Yucca 017,57159,Valid,H5,200.0,Found,2011,34.81923,-114.27735,"(34.81923, -114.27735)",Mohave County,Arizona,United States
45693,Zaffra,30383,Valid,"Iron, IAB-MG",3000.0,Found,1919,35.00000,-94.75000,"(35.0, -94.75)",Le Flore County,Oklahoma,United States
45703,Zenda,30400,Valid,"Iron, IAB complex",3700.0,Found,1955,42.51333,-88.48944,"(42.51333, -88.48944)",Walworth County,Wisconsin,United States


In [124]:
# Create a scatter map using the 'reclat' and 'reclong' columns of the dataframe as latitude and longitude, respectively
fig = px.scatter_mapbox(us_meteor_data, lat="reclat", lon="reclong")

# customize the layout of the Map
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(center=dict(lat=0, lon=0), zoom=0),
    margin=dict(l=0, r=0, t=0, b=0),
    width=600, height=600
)

# Display the map
fig.show()

In [ ]:
# save US data with county info to csv
us_meteor_data.to_csv('Resources/Meteorite_Landings_US_county.csv', index=False)

In [127]:
us_meteor_data['elevation'] = 0.0

In [128]:
us_meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country,elevation
28,Allegan,2276,Valid,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",Allegan County,Michigan,United States,0.0
34,Andover,2295,Valid,L6,3200.0,Fell,1898,44.61667,-70.75000,"(44.61667, -70.75)",Oxford County,Maine,United States,0.0
46,Archie,2329,Valid,H6,5070.0,Fell,1932,38.50000,-94.30000,"(38.5, -94.3)",Cass County,Missouri,United States,0.0
49,Ash Creek,48954,Valid,L6,9500.0,Fell,2009,31.80500,-97.01000,"(31.805, -97.01)",McLennan County,Texas,United States,0.0
54,Athens,4885,Valid,LL6,265.0,Fell,1933,34.75000,-87.00000,"(34.75, -87.0)",Limestone County,Alabama,United States,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45688,Yucca 016,57158,Valid,H5,25.9,Found,2011,34.82658,-114.27763,"(34.82658, -114.27763)",Mohave County,Arizona,United States,0.0
45689,Yucca 017,57159,Valid,H5,200.0,Found,2011,34.81923,-114.27735,"(34.81923, -114.27735)",Mohave County,Arizona,United States,0.0
45693,Zaffra,30383,Valid,"Iron, IAB-MG",3000.0,Found,1919,35.00000,-94.75000,"(35.0, -94.75)",Le Flore County,Oklahoma,United States,0.0
45703,Zenda,30400,Valid,"Iron, IAB complex",3700.0,Found,1955,42.51333,-88.48944,"(42.51333, -88.48944)",Walworth County,Wisconsin,United States,0.0


In [129]:
qlats = us_meteor_data['reclat'].to_list()
print(type(qlats))
qlats

<class 'list'>


[42.53333,
 44.61667,
 38.5,
 31.805,
 34.75,
 34.31667,
 36.8,
 35.96667,
 34.5,
 45.41667,
 38.25,
 40.66813,
 36.75,
 39.8,
 33.6,
 39.08333,
 40.30583,
 42.53333,
 34.16667,
 40.91667,
 36.83333,
 31.83333,
 40.48,
 40.5,
 37.62194,
 35.5,
 38.47028,
 37.26667,
 36.08333,
 44.38333,
 36.5,
 43.2,
 36.16667,
 35.03333,
 40.93333,
 32.1025,
 44.9,
 43.76111,
 35.95,
 35.63333,
 36.83333,
 38.4,
 34.4,
 40.25,
 39.7825,
 36.4,
 40.78167,
 39.24667,
 43.41667,
 39.75,
 35.55,
 36.05,
 32.53333,
 36.1,
 47.81667,
 30.83333,
 43.25,
 36.78333,
 33.01667,
 34.48333,
 41.68333,
 41.38333,
 32.675,
 38.25,
 34.9,
 41.8,
 21.3,
 39.35,
 39.08333,
 40.35,
 29.45,
 43.58333,
 30.75,
 35.88333,
 34.58333,
 37.91667,
 38.70066,
 36.00833,
 38.25,
 32.03333,
 32.21667,
 42.71667,
 41.9,
 35.8,
 42.9075,
 35.4,
 38.5,
 31.60833,
 35.25,
 35.41667,
 36.6,
 38.41667,
 40.0,
 29.94718,
 44.08333,
 40.08528,
 36.21667,
 39.68333,
 28.5475,
 38.6,
 39.8,
 21.3,
 36.06667,
 41.48472,
 36.21667,
 41.2833

In [130]:
qlongs = us_meteor_data['reclong'].to_list()
print(type(qlongs))
qlongs

<class 'list'>


[-85.88333,
 -70.75,
 -94.3,
 -97.01,
 -87.0,
 -96.15,
 -108.0,
 -82.48333,
 -88.66667,
 -98.31667,
 -83.75,
 -117.18913,
 -93.5,
 -101.2,
 -96.46667,
 -89.15,
 -105.02325,
 -73.83333,
 -80.28333,
 -78.08333,
 -97.33333,
 -98.83333,
 -89.00417,
 -80.08333,
 -82.23722,
 -93.5,
 -105.24139,
 -89.58333,
 -78.06667,
 -68.75,
 -79.25,
 -96.91667,
 -87.33333,
 -81.88333,
 -79.73333,
 -81.87278,
 -90.28333,
 -84.50778,
 -97.58333,
 -78.13333,
 -84.35,
 -84.25,
 -87.06667,
 -74.0,
 -104.93056,
 -86.5,
 -96.47167,
 -104.58817,
 -94.83333,
 -97.03333,
 -77.53333,
 -94.16667,
 -87.16667,
 -81.41667,
 -96.85,
 -97.76667,
 -93.66667,
 -78.08333,
 -83.96667,
 -87.83333,
 -112.13333,
 -86.6,
 -94.51167,
 -86.16667,
 -110.18333,
 -91.86667,
 -157.86667,
 -100.45,
 -94.4,
 -104.9,
 -96.0,
 -89.6,
 -95.95,
 -99.33333,
 -87.5,
 -92.08333,
 -77.21163,
 -95.15,
 -85.75,
 -84.76667,
 -104.0,
 -99.38333,
 -91.6,
 -84.1,
 -90.36556,
 -92.05,
 -101.1,
 -102.85833,
 -80.5,
 -79.38333,
 -88.1,
 -77.16667,
 -81.7

In [131]:
# Dependencies
import requests

# url = f"https://api.opentopodata.org/v1/etopo1?locations={qlat},{qlong}"

# qlats = [42.53333, 26.96667, 44.61667, 38.5]
# qlongs = [-85.88333, -105.31667, -70.75, -94.3]

qlatlong = []

for x in range(0,len(qlats)):
    strlalo= f'{qlats[x]},{qlongs[x]}'
    qlatlong.append(strlalo)
    print(strlalo)
    type(strlalo)

print(qlatlong)


responses = []

for qlat in qlatlong:
    print(qlat)
    url = f"https://api.opentopodata.org/v1/etopo1?locations={qlat}"
    movie_data = requests.get(url).json()
    responses.append(movie_data)
    print(movie_data)

print(responses)

42.53333,-85.88333
44.61667,-70.75
38.5,-94.3
31.805,-97.01
34.75,-87.0
34.31667,-96.15
36.8,-108.0
35.96667,-82.48333
34.5,-88.66667
45.41667,-98.31667
38.25,-83.75
40.66813,-117.18913
36.75,-93.5
39.8,-101.2
33.6,-96.46667
39.08333,-89.15
40.30583,-105.02325
42.53333,-73.83333
34.16667,-80.28333
40.91667,-78.08333
36.83333,-97.33333
31.83333,-98.83333
40.48,-89.00417
40.5,-80.08333
37.62194,-82.23722
35.5,-93.5
38.47028,-105.24139
37.26667,-89.58333
36.08333,-78.06667
44.38333,-68.75
36.5,-79.25
43.2,-96.91667
36.16667,-87.33333
35.03333,-81.88333
40.93333,-79.73333
32.1025,-81.87278
44.9,-90.28333
43.76111,-84.50778
35.95,-97.58333
35.63333,-78.13333
36.83333,-84.35
38.4,-84.25
34.4,-87.06667
40.25,-74.0
39.7825,-104.93056
36.4,-86.5
40.78167,-96.47167
39.24667,-104.58817
43.41667,-94.83333
39.75,-97.03333
35.55,-77.53333
36.05,-94.16667
32.53333,-87.16667
36.1,-81.41667
47.81667,-96.85
30.83333,-97.76667
43.25,-93.66667
36.78333,-78.08333
33.01667,-83.96667
34.48333,-87.83333
41.68

In [132]:
responses[0]['results'][0]['elevation']

225.0

In [133]:
elevations = []

for ellie in range (0,len(responses)):
    elevations.append(responses[ellie]['results'][0]['elevation'])

elevations


[225.0,
 220.0,
 264.0,
 170.0,
 204.0,
 201.0,
 1758.0,
 1166.0,
 119.0,
 394.0,
 279.0,
 1852.0,
 294.0,
 931.0,
 216.0,
 179.0,
 1510.0,
 45.0,
 65.0,
 616.0,
 325.0,
 534.0,
 243.0,
 230.0,
 363.0,
 172.0,
 1689.0,
 123.0,
 88.0,
 25.0,
 162.0,
 374.0,
 215.0,
 257.0,
 380.0,
 55.0,
 409.0,
 218.0,
 342.0,
 59.0,
 322.0,
 249.0,
 182.0,
 10.0,
 1600.0,
 162.0,
 405.0,
 2089.0,
 396.0,
 433.0,
 20.0,
 382.0,
 52.0,
 461.0,
 254.0,
 310.0,
 378.0,
 91.0,
 180.0,
 213.0,
 1307.0,
 208.0,
 93.0,
 211.0,
 1544.0,
 219.0,
 2.0,
 823.0,
 292.0,
 1458.0,
 31.0,
 244.0,
 96.0,
 627.0,
 188.0,
 281.0,
 27.0,
 255.0,
 144.0,
 149.0,
 892.0,
 649.0,
 243.0,
 282.0,
 331.0,
 257.0,
 959.0,
 812.0,
 164.0,
 98.0,
 144.0,
 24.0,
 297.0,
 4.0,
 44.0,
 252.0,
 158.0,
 715.0,
 36.0,
 278.0,
 179.0,
 240.0,
 87.0,
 218.0,
 1525.0,
 81.0,
 1220.0,
 237.0,
 636.0,
 118.0,
 85.0,
 1221.0,
 295.0,
 2473.0,
 974.0,
 747.0,
 497.0,
 59.0,
 235.0,
 387.0,
 57.0,
 77.0,
 112.0,
 76.0,
 5.0,
 1381.0,
 147.0,


In [137]:
print(type(elevations))
print(len(elevations))
len(us_meteor_data)

<class 'list'>
1650


1650

In [142]:
us_meteor_data['elevation'] = elevations
us_meteor_data

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,county,state,country,elevation
28,Allegan,2276,Valid,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",Allegan County,Michigan,United States,225.0
34,Andover,2295,Valid,L6,3200.0,Fell,1898,44.61667,-70.75000,"(44.61667, -70.75)",Oxford County,Maine,United States,220.0
46,Archie,2329,Valid,H6,5070.0,Fell,1932,38.50000,-94.30000,"(38.5, -94.3)",Cass County,Missouri,United States,264.0
49,Ash Creek,48954,Valid,L6,9500.0,Fell,2009,31.80500,-97.01000,"(31.805, -97.01)",McLennan County,Texas,United States,170.0
54,Athens,4885,Valid,LL6,265.0,Fell,1933,34.75000,-87.00000,"(34.75, -87.0)",Limestone County,Alabama,United States,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45688,Yucca 016,57158,Valid,H5,25.9,Found,2011,34.82658,-114.27763,"(34.82658, -114.27763)",Mohave County,Arizona,United States,547.0
45689,Yucca 017,57159,Valid,H5,200.0,Found,2011,34.81923,-114.27735,"(34.81923, -114.27735)",Mohave County,Arizona,United States,536.0
45693,Zaffra,30383,Valid,"Iron, IAB-MG",3000.0,Found,1919,35.00000,-94.75000,"(35.0, -94.75)",Le Flore County,Oklahoma,United States,196.0
45703,Zenda,30400,Valid,"Iron, IAB complex",3700.0,Found,1955,42.51333,-88.48944,"(42.51333, -88.48944)",Walworth County,Wisconsin,United States,304.0


In [143]:
# save US data with county info to csv
us_meteor_data.to_csv('Resources/Meteorite_US_county_elev.csv')

In [144]:
lcounty = us_meteor_data['county'].to_list()
print(len(lcounty))
lcounty

1650


['Allegan County',
 'Oxford County',
 'Cass County',
 'McLennan County',
 'Limestone County',
 'Atoka County',
 'San Juan County',
 'Yancey County',
 'Lee County',
 'Brown County',
 'Bath County',
 'Humboldt County',
 'Stone County',
 'Rawlins County',
 'Grayson County',
 'Fayette County',
 'Weld County',
 'Albany County',
 'Lee County',
 'Centre County',
 'Kay County',
 'Brown County',
 'McLean County',
 'Allegheny County',
 'Pike County',
 'Johnson County',
 'Fremont County',
 'Cape Girardeau County',
 'Nash County',
 'Hancock County',
 'Caswell County',
 'Lincoln County',
 'Dickson County',
 'Spartanburg County',
 'Butler County',
 'Evans County',
 'Marathon County',
 'Midland County',
 'Logan County',
 'Wilson County',
 'McCreary County',
 'Harrison County',
 'Morgan County',
 'Monmouth County',
 '',
 'Sumner County',
 'Lancaster County',
 'Elbert County',
 'Emmet County',
 'Washington County',
 'Pitt County',
 'Washington County',
 'Perry County',
 'Wilkes County',
 'Polk County',

NEXT - Create CSV with COUNTY LAND COVER INFO and bring in to this nb.
consider scraping https://www.mrlc.gov/eva/ using lcounty